## --- Day 10: Pipe Maze ---
You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

.....
.F-7.
.|.|.
.L-J.
.....
If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

.....
.S-7.
.|.|.
.L-J.
.....
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

-L|F7
7S-7|
L|7||
-L-J|
L|-JF
In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...
Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

.....
.S-7.
.|.|.
.L-J.
.....
You can count the distance each tile in the loop is from the starting point like this:

.....
.012.
.1.3.
.234.
.....
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...
Here are the distances for each tile on that loop:

..45.
.236.
01.78
14567
23...
Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [1]:
with open('../inputs/day-10.txt') as f:
    raw_input = f.read()

In [27]:
raw_example1 = '''.....
.S-7.
.|.|.
.L-J.
.....'''
raw_example2 = '''7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ'''

In [28]:
def pre_process(raw_input):
    return [[c for c in line] for line in raw_input.split('\n')]
example1 = pre_process(raw_example1)
example2 = pre_process(raw_example2)
example1

[['.', '.', '.', '.', '.'],
 ['.', 'S', '-', '7', '.'],
 ['.', '|', '.', '|', '.'],
 ['.', 'L', '-', 'J', '.'],
 ['.', '.', '.', '.', '.']]

In [49]:
def find_starting_point(grid):
    for l, line in enumerate(grid):
        for c, col in enumerate(line):
            if col == 'S':
                return l, c
            
find_starting_point(pre_process(raw_example1))

(1, 1)

In [93]:
def find_starting_neighbors(grid, starting_point):
    l, c = starting_point
    neighbors = []
    if grid[l - 1][c] in '|7F': # Si arriba (-1, 0) es |, 7 o F
        neighbors.append((l - 1, c))
    if grid[l + 1][c] in '|JL': # Si abajo (1, 0) es |, J o L
        neighbors.append((l + 1, c))
    if grid[l][c + 1] in '-7J': # Si derecha (0, 1) es -, 7 o J 
        neighbors.append((l, c + 1))
    if grid[l][c - 1] in '-FL': # Si izquierda (0, -1) es -, F o L
        neighbors.append((l, c - 1))

    return neighbors

def find_next_pipe(grid, current_pipe_loc, previous_pipe_loc):
    l, c = current_pipe_loc
    current_pipe = grid[l][c]
    next_pipe = []
    if current_pipe in '|7F': # Conecta hacia abajo
        next_pipe.append((l+1, c))
    if current_pipe in '|JL': # Conecta hacia arriba
        next_pipe.append((l-1, c))
    if current_pipe in '-7J': # Conecta hacia la izquierda
        next_pipe.append((l, c-1))
    if current_pipe in '-FL': # Conecta hacia la derecha
        next_pipe.append((l, c+1))
    
    next_pipe.remove(previous_pipe_loc)
    return next_pipe[0]



In [99]:
def follow_pipes(grid):
    starting_point = find_starting_point(grid)
    starting_neighbors = find_starting_neighbors(grid, starting_point)
    
    new_locs = [n for n in starting_neighbors]
    previous_locs = [starting_point, starting_point]
    counter = 1
    while new_locs.count(new_locs[0]) < 2:
        new_neighbours = [find_next_pipe(grid, loc, prev_loc) for loc, prev_loc in zip(new_locs, previous_locs)]
        previous_locs = [l for l in new_locs]  # get new previous locs
        new_locs = [l for l in new_neighbours] # get new locs
        counter += 1 
        
    return counter

In [101]:
follow_pipes(example1)

4

In [102]:
problem_input = pre_process(raw_input)
follow_pipes(problem_input)

7066

--- Part Two ---
You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
Here are just the tiles that are enclosed by the loop marked with I:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?

In [105]:
def follow_and_save_pipes(grid):
    starting_point = find_starting_point(grid)
    starting_neighbors = find_starting_neighbors(grid, starting_point)
    total_locs = [starting_point, *starting_neighbors]
    new_locs = [n for n in starting_neighbors]
    previous_locs = [starting_point, starting_point]
    counter = 1
    while new_locs.count(new_locs[0]) < 2:
        new_neighbours = [find_next_pipe(grid, loc, prev_loc) for loc, prev_loc in zip(new_locs, previous_locs)]
        previous_locs = [l for l in new_locs]  # get new previous locs
        new_locs = [l for l in new_neighbours] # get new locs
        total_locs.extend(new_locs)
        counter += 1 
        
    return total_locs

In [107]:
example2

[['7', '-', 'F', '7', '-'],
 ['.', 'F', 'J', '|', '7'],
 ['S', 'J', 'L', 'L', '7'],
 ['|', 'F', '-', '-', 'J'],
 ['L', 'J', '.', 'L', 'J']]

In [106]:
follow_and_save_pipes(example2)

[(2, 0),
 (3, 0),
 (2, 1),
 (4, 0),
 (1, 1),
 (4, 1),
 (1, 2),
 (3, 1),
 (0, 2),
 (3, 2),
 (0, 3),
 (3, 3),
 (1, 3),
 (3, 4),
 (2, 3),
 (2, 4),
 (2, 4)]

In [183]:
def check_if_in(grid, loc, pipe_loop_locs, up = True):
    l, c = loc
    if grid[l][c] in pipe_loop_locs:
        return False
    counter = 0
    way = range(0,l) if up else range(l, len(grid))
    for line in way: # recorro hacia arriba o abajo
        if (line, c) in pipe_loop_locs:
            if grid[line][c] in '-FL': # cuento si es un pipe que conecta hacia la derecha
                counter += 1 # cuento cada vez que cruzo el loop
    return (counter % 2 != 0)

In [169]:
from tqdm import tqdm

In [189]:
def get_status_for_loc(grid):
    counter = 0
    len_grid = len(grid)
    loop_locs = follow_and_save_pipes(grid)
    for l in tqdm(range(len_grid), total = len_grid, desc = 'l: '):
        len_line = range(len(grid[l]))
        for c in len_line:
            if (l,c) not in loop_locs:
                if l < len_grid / 2:
                    counter += check_if_in(grid, (l,c), loop_locs, up = True)
                else:
                    counter += check_if_in(grid, (l,c), loop_locs, up = False)
    return counter

In [196]:
example3_raw = '''.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...'''
example3 = pre_process(example3_raw)
r = get_status_for_loc(example3)
r

l: 100%|██████████| 10/10 [00:00<00:00, 8912.67it/s]


8

In [190]:
s = get_status_for_loc(problem_input)

l:   0%|          | 0/141 [00:00<?, ?it/s]

l: 100%|██████████| 141/141 [00:35<00:00,  4.01it/s]


In [191]:
s

401